# Convolutional Neural Networks (CNNs)

## Quelle der Daten

https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database?resource=download (zuletzt aufgerufen 01/2024)
    
M.E.H. Chowdhury, T. Rahman, A. Khandakar, R. Mazhar, M.A. Kadir, Z.B. Mahbub, K.R. Islam, M.S. Khan, A. Iqbal, N. Al-Emadi, M.B.I. Reaz, M. T. Islam, “Can AI help in screening Viral and COVID-19 pneumonia?” IEEE Access, Vol. 8, 2020, pp. 132665 - 132676.

Rahman, T., Khandakar, A., Qiblawey, Y., Tahir, A., Kiranyaz, S., Kashem, S.B.A., Islam, M.T., Maadeed, S.A., Zughaier, S.M., Khan, M.S. and Chowdhury, M.E., 2020. Exploring the Effect of Image Enhancement Techniques on COVID-19 Detection using Chest X-ray Images.

## Installation der Bibliotheken

In [ ]:
import os

import cv2

import matplotlib.pyplot as plt

import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

## Einlesen und Präprozessierung der Daten

In [ ]:
main_path = "C:\\Users\\Tim\\Desktop\\Buchprojekt KI Medizin\\Programmierung\\COVID-19_Radiography_Dataset"

covid_dir = os.path.join(main_path, "covid_images")
normal_dir = os.path.join(main_path, "normal_images")

print("Anzahl Bilder mit COVID:", len(os.listdir(covid_dir)))
print("Anzahl normaler Bilder:", len(os.listdir(normal_dir)))

In [ ]:
example_image = cv2.imread(os.path.join(covid_dir, "COVID-1.png"))

plt.imshow(example_image)

In [ ]:
example_image.shape

In [ ]:
def loadImages(dir, size, label):
  images = []
  labels = []
    
  for i in range(len(size)):
    img_path = dir + "/" + size[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(label)
  
  images = np.asarray(images)
    
  return images, labels

In [ ]:
covid_images, covid_labels = loadImages(covid_dir, os.listdir(covid_dir), 1)

len(covid_images), len(covid_labels)

In [ ]:
normal_images, normal_labels = loadImages(normal_dir, os.listdir(normal_dir), 0)

len(normal_images), len(normal_labels)

## Aufteilung der Daten

In [ ]:
x = np.r_[covid_images, normal_images]

y = np.r_[covid_labels, normal_labels]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

## CNN-Architektur

In [ ]:
print(x_test[0].shape)

In [ ]:
model = Sequential()

model.add(Conv2D(32, 3, input_shape = (100, 100, 3), activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16, 3, activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16, 3, activation = "relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

model.summary()

## Modelltraining und -evaluation

In [ ]:
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = "accuracy")

In [ ]:
model.fit(x_train, y_train, 
          batch_size = 32, epochs = 5, 
          validation_data = (x_test, y_test))

In [ ]:
plt.plot(model.history.history["accuracy"], label = "train_accuracy")
plt.plot(model.history.history["val_accuracy"], label = "test_accuracy")
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)

y_pred_bin = (y_pred > 0.5).astype(int)

In [ ]:
print("Label:", y_test[0])

print("Modell-Ausgabe:", y_pred_bin[0])

plt.imshow(x_test[0])